# One classifier each name: OCEN (Name group)
This method throw away the record below threshold.

Also this file evaluate different embedding method with OCEN.
1. Text embedding only
2. Citation embedding only
3. Combined embedding

In [1]:
import os
import sys
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import com_func

# parameters
#----- threshold for selecting set of name group -----------#
threshold_select_name_group = 100
#----- threshold for selecting min sample in name group ----#
threshold_lower = 100
threshold_upper = 110

Dataset = "pubmed"

In [2]:
# text embedding only
pp_text_emb = ["tf", "tf_idf", "lsa", "pv_dm", "pv_dbow"]
pp_citation_emb = ["off"]

In [5]:
# citation embedding only
pp_text_emb = ["off"]
pp_citation_emb = ["node2vec", "n2v"]

In [2]:
# combined embedding
pp_text_emb = ["lsa", "pv_dm", "pv_dbow"]
pp_citation_emb = ["n2v"]

In [6]:
print(pp_text_emb)
print(pp_citation_emb)

['off']
['node2vec', 'n2v']


In [7]:
# load the file
import io
import collections
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from statistics import mean 

# fix random seed for reproducibility
np.random.seed(1)

fileDir = "../Data/"+Dataset+"/canopies_labeled/"
listfiles = os.listdir(fileDir)

lr_diff_embedding_result = []
svm_diff_embedding_result = []

# ----------------------- different textual embedding ----------------------#
for text_emb in pp_text_emb:
    print("Load text embedding: ", text_emb)
    # read pretrained embeddings
    all_textual_embedding, all_textual_emb_pid = com_func.read_text_embedding(emb_type=text_emb, training_size="140k")
    
    for citation_emb in pp_citation_emb:
        print("Load citation embedding: ", citation_emb)
        all_citation_embedding = com_func.read_citation_embedding_sorted(emb_type = citation_emb)
        all_citation_emb_pid = [emb[0] for emb in all_citation_embedding]
        all_citation_embedding = [emb[1:] for emb in all_citation_embedding]
        
        threshold_change_all_lr_f1s = []
        threshold_change_all_svm_f1s = []
        threshold_change = []

        # -------------- different threshold (step by 10) -----------------------#
        for step_threshold in range(threshold_lower, threshold_upper, 10):
            threshold_change.append(step_threshold)
            # collect statistic to output
            allname, num_class, per_class_count, selected_sample_size, orginal_sample_size= ([] for i in range(5))

            all_mnb_accuracy, all_mnb_f1, all_LR_accuracy = ([] for i in range(3))
            all_LR_f1, all_svcLinear_accuracy, all_svcLinear_f1 = ([] for i in range(3))

            total_selected_group = 0

            # ------- different name group in all name group --------------------#
            for file in listfiles:
                # group name
                temp = file.split("_")
                name = temp[1]+"_"+temp[-1]
                print("For name: ",name)
                # read needed content in labeled file
                labeled_data = com_func.read_pid_aid(fileDir+file)
                #----------- select name group contain productive author------------------------------------#
                #----------- (contain pair of author write more than 100 papers) ---------------------------#
                # count number of paper each author write based on author ID
                authorCounter = collections.Counter(labeled_data["authorID"])
                # remove name group that do not contain pair of author write more than 100 papers
                for k in list(authorCounter):
                    if authorCounter[k] < threshold_select_name_group:
                        del authorCounter[k]
                # if only have one class or no class pass the threshold, not applicable
                if(len(authorCounter)==0) or (len(authorCounter)==1):
                    print(name, " pass")
                else:
                    total_selected_group+= 1
                    orginal_sample_size.append(len(labeled_data))
                    #--------select authors in name group are very productive (more than threshold)---------#
                    print("Total sample size before apply threshold: ",len(labeled_data))
                    # count number of paper each author write based on author ID
                    paperCounter = collections.Counter(labeled_data["authorID"])
                    print(paperCounter)
                    print("Total author before apply threshoid: ", len(paperCounter))
                    # collect per class statistic
                    for k in list(paperCounter):
                        if paperCounter[k] < step_threshold:
                            del paperCounter[k]
                    temp =list(paperCounter.keys())
                    print(temp)
                    print("Total author after apply threshoid: ", len(temp))
                    # remove samples that are smaller than threshold
                    labeled_data = labeled_data[labeled_data.authorID.isin(temp)]
                    print("Total sample size after apply threshold: ",len(labeled_data))
                    selected_sample_size.append(len(labeled_data))
                    allname.append(name)
                    num_class.append(len(paperCounter))
                    per_class_count.append(paperCounter)
                    #------------ extract paper representation -------------------------------------------#
                    # shuffle the data
                    labeled_data = labeled_data.sample(frac=1).reset_index(drop=True)
                    # extract true label and pid
                    label = labeled_data["authorID"]
                    pid = labeled_data["paperID"]
                    # list of different data field
                    part_collection = []
                    # select feature wanted to fit to clustering/classification algorithm
                    # data part, textual information
                    data_part_textual = com_func.extract_embedding(all_textual_embedding, all_textual_emb_pid, pid)
                    print("Text embedding shape: ", data_part_textual.shape)
                    part_collection.append(data_part_textual)
                    # data part, citation information
                    data_part_citation = com_func.extract_embedding(all_citation_embedding, all_citation_emb_pid, pid)
                    print("Citation embedding shape: ", data_part_citation.shape)
                    part_collection.append(data_part_citation)
                    # merge different part of data data together by concatenate it all together
                    # remove empty emb (when emb set off)
                    part_collection = [part for part in part_collection if len(part)!=0]
                    if len(part_collection)>1:
                        combinedata = np.concatenate(part_collection,axis=1)
                    elif len(part_collection)==1:
                        if isinstance(part_collection[0], pd.DataFrame):
                            combinedata = part_collection[0].values
                        else:
                            combinedata = part_collection[0]
                    else:
                        print("No data available")
                        break
                    print("Final feature (combined embedding) shape: ", combinedata.shape)
                    # -------------- using converted feature vector to train classifier-------------------#
                    if text_emb == "tf":
                        # using multinomial naive bayes
                        clf = MultinomialNB()
                        mnbaccuracy, mnbmarcof1, tp, tn, fp, fn = com_func.k_fold_cv_with_accumulate_statistic(combinedata, label, clf, k=10)
                        print("MNB Accuracy: ",mnbaccuracy)
                        print("MNB F1: ", mnbmarcof1)
                        all_mnb_accuracy.append(mnbaccuracy)
                        all_mnb_f1.append(mnbmarcof1)
                    # using logistic regression
                    clf = LogisticRegression(multi_class='ovr')
                    LRaccuracy, LRmarcof1, tp, tn, fp, fn = com_func.k_fold_cv_with_accumulate_statistic(combinedata, label, clf, k=10)
                    print("LR Accuracy: ",LRaccuracy)
                    print("LR F1: ", LRmarcof1)
                    all_LR_accuracy.append(LRaccuracy)
                    all_LR_f1.append(LRmarcof1)
                    # using SVM with linear kernal
                    clf = SVC(decision_function_shape='ovr', kernel='linear')
                    svcaccuracy, svcmarcof1, tp, tn, fp, fn = com_func.k_fold_cv_with_accumulate_statistic(combinedata, label, clf, k=10)
                    print("svc Accuracy: ",svcaccuracy)
                    print("svc F1: ", svcmarcof1)
                    all_svcLinear_accuracy.append(svcaccuracy)
                    all_svcLinear_f1.append(svcmarcof1)
                    
            if text_emb == "tf":
                # write evaluation result to excel
                output = pd.DataFrame({'Name Group':allname,"Class number":num_class,"Per class size":per_class_count, 
                                       "Total selected sample size":selected_sample_size,"Orginal sample size":orginal_sample_size,
                                       "MNB Accuracy":all_mnb_accuracy, "MNB macro F1": all_mnb_f1, 
                                       "svc(linear) accuracy":all_svcLinear_accuracy, "svc(linear) macro f1": all_svcLinear_f1, 
                                       "logistic regression accuracy":all_LR_accuracy, "logistic regression macro f1": all_LR_f1})
            else:
                # write evaluation result to excel
                output = pd.DataFrame({'Name Group':allname, "Class number":num_class, "Per class size":per_class_count,
                                       "Total selected sample size":selected_sample_size,"Orginal sample size":orginal_sample_size,
                                       "svc(linear) accuracy":all_svcLinear_accuracy, "svc(linear) macro f1": all_svcLinear_f1, 
                                       "logistic regression accuracy":all_LR_accuracy, "logistic regression macro f1": all_LR_f1})

            savePath = "../../result/"+Dataset+"/OCEN_global_emb_sample=140k/"
            filename = "(Global emb sample 140k) citation="+citation_emb+"_textual="+text_emb+"_threshold="+str(step_threshold)+"_namegroupcount="+str(total_selected_group)+".csv"
            com_func.write_csv_df(savePath, filename, output)
            print("Done")

            threshold_change_all_lr_f1s.append(all_LR_f1)
            threshold_change_all_svm_f1s.append(all_svcLinear_f1)

        lr_diff_embedding_result.append(threshold_change_all_lr_f1s)
        svm_diff_embedding_result.append(threshold_change_all_svm_f1s)

Load text embedding:  off
Load citation embedding:  node2vec
Total citation vector records: 117857
Vector dimension:  101
For name:  j_read
j_read  pass
For name:  f_esteves
f_esteves  pass
For name:  c_miller
c_miller  pass
For name:  r_jha
r_jha  pass
For name:  a_lowe
a_lowe  pass
For name:  a_vega
a_vega  pass
For name:  k_smith
k_smith  pass
For name:  j_gordon
j_gordon  pass
For name:  s_liao
s_liao  pass
For name:  j_qian
j_qian  pass
For name:  s_bernardi
s_bernardi  pass
For name:  t_hill
t_hill  pass
For name:  s_schindler
s_schindler  pass
For name:  j_williams
j_williams  pass
For name:  s_jacobson
s_jacobson  pass
For name:  e_andrade
e_andrade  pass
For name:  t_santos
t_santos  pass
For name:  k_kim
Total sample size before apply threshold:  1111
Counter({'0000-0002-6929-5359': 211, '0000-0001-9498-284X': 154, '0000-0002-5878-8895': 139, '0000-0002-1864-3392': 92, '0000-0002-7045-8004': 57, '0000-0001-7896-6751': 57, '0000-0002-7991-9428': 55, '0000-0002-4010-1063': 45, 

For name:  m_hidalgo
m_hidalgo  pass
For name:  k_jacobsen
k_jacobsen  pass
For name:  s_kelly
s_kelly  pass
For name:  s_james
s_james  pass
For name:  p_persson
p_persson  pass
For name:  y_tanaka
y_tanaka  pass
For name:  c_gao
c_gao  pass
For name:  w_jung
w_jung  pass
For name:  s_lewis
s_lewis  pass
For name:  w_han
w_han  pass
For name:  m_shah
m_shah  pass
For name:  c_arango
c_arango  pass
For name:  r_young
r_young  pass
For name:  r_coleman
r_coleman  pass
For name:  b_kang
b_kang  pass
For name:  s_carter
s_carter  pass
For name:  c_thomas
c_thomas  pass
For name:  m_gutierrez
m_gutierrez  pass
For name:  s_moon
s_moon  pass
For name:  r_pereira
r_pereira  pass
For name:  a_nielsen
a_nielsen  pass
For name:  j_conde
j_conde  pass
For name:  k_wright
k_wright  pass
For name:  m_parker
m_parker  pass
For name:  h_huang
h_huang  pass
For name:  j_terry
j_terry  pass
For name:  y_xu
y_xu  pass
For name:  a_melo
a_melo  pass
For name:  r_doyle
r_doyle  pass
For name:  m_bernardo

Missing Sample:  25561562
Missing Sample:  18424550
Missing Sample:  18183207
Missing Sample:  19547208
Missing Sample:  19516457
Missing Sample:  18071574
Missing Sample:  16373528
Missing Sample:  15483720
Missing Sample:  24515200
Missing Sample:  18542306
Missing Sample:  24829151
Missing Sample:  19436371
Missing Sample:  14605412
Missing Sample:  24021215
Missing Sample:  18049570
Missing Sample:  21418014
Missing Sample:  18542531
Missing Sample:  19516664
Missing Sample:  19532763
Missing Sample:  19475054
Missing Sample:  19532619
Total missing sample:  73
Citation embedding shape:  (398, 100)
Final feature (combined embedding) shape:  (398, 100)
Binary case
                     precision    recall  f1-score   support

0000-0002-7751-1058       0.84      0.94      0.89       167
0000-0003-0960-6415       0.95      0.87      0.91       231

          micro avg       0.90      0.90      0.90       398
          macro avg       0.90      0.91      0.90       398
       weighted a

Binary case
                     precision    recall  f1-score   support

0000-0001-8592-0698       0.89      0.98      0.93       121
0000-0003-0852-0767       0.98      0.87      0.92       117

          micro avg       0.93      0.93      0.93       238
          macro avg       0.93      0.93      0.93       238
       weighted avg       0.93      0.93      0.93       238

TP:  102 TN:  119 FP:  2 FN:  15
LR Accuracy:  0.9285714285714286
LR F1:  0.9282051282051282
Binary case
                     precision    recall  f1-score   support

0000-0001-8592-0698       0.89      0.96      0.92       121
0000-0003-0852-0767       0.95      0.87      0.91       117

          micro avg       0.92      0.92      0.92       238
          macro avg       0.92      0.92      0.92       238
       weighted avg       0.92      0.92      0.92       238

TP:  102 TN:  116 FP:  5 FN:  15
svc Accuracy:  0.9159663865546218
svc F1:  0.9156746031746033
For name:  j_gao
j_gao  pass
For name:  d_fernande

l_campos  pass
For name:  m_iqbal
m_iqbal  pass
For name:  s_lim
s_lim  pass
For name:  p_li
p_li  pass
For name:  f_andrade
f_andrade  pass
For name:  c_schmitt
c_schmitt  pass
For name:  t_tan
t_tan  pass
For name:  h_gomes
h_gomes  pass
For name:  m_matos
m_matos  pass
For name:  k_ryan
k_ryan  pass
For name:  w_zheng
w_zheng  pass
For name:  j_franco
j_franco  pass
For name:  l_walker
l_walker  pass
For name:  a_gordon
a_gordon  pass
For name:  z_yin
z_yin  pass
For name:  c_gu
c_gu  pass
For name:  a_soto
a_soto  pass
For name:  h_hsieh
h_hsieh  pass
For name:  m_crespo
m_crespo  pass
For name:  s_phillips
s_phillips  pass
For name:  r_rodrigues
r_rodrigues  pass
For name:  a_mansour
a_mansour  pass
For name:  a_lau
a_lau  pass
For name:  j_berg
j_berg  pass
For name:  l_wilson
l_wilson  pass
For name:  c_park
c_park  pass
For name:  r_thomas
r_thomas  pass
For name:  j_fonseca
j_fonseca  pass
For name:  s_henderson
s_henderson  pass
For name:  m_coelho
m_coelho  pass
For name:  j

...

p_thompson  pass
For name:  a_castro
a_castro  pass
For name:  j_zhang
j_zhang  pass
For name:  j_rodrigues
j_rodrigues  pass
For name:  s_watson
s_watson  pass
For name:  c_barros
c_barros  pass
For name:  f_cardoso
f_cardoso  pass
For name:  m_pinto
m_pinto  pass
For name:  j_cuevas
j_cuevas  pass
For name:  j_chang
j_chang  pass
For name:  a_dias
a_dias  pass
For name:  j_choi
j_choi  pass
For name:  m_ahmed
m_ahmed  pass
For name:  j_jo
j_jo  pass
For name:  n_dawson
n_dawson  pass
For name:  j_barbosa
j_barbosa  pass
For name:  e_o'connor
e_o'connor  pass
For name:  c_zheng
c_zheng  pass
For name:  r_hall
r_hall  pass
For name:  d_hwang
d_hwang  pass
For name:  c_shen
c_shen  pass
For name:  v_lopes
v_lopes  pass
For name:  m_quintana
m_quintana  pass
For name:  j_nunes
j_nunes  pass
For name:  z_nagy
z_nagy  pass
For name:  e_brown
e_brown  pass
For name:  j_nielsen
Total sample size before apply threshold:  913
Counter({'0000-0002-9955-6003': 487, '0000-0001-5568-2916': 105, '00

s_hong  pass
For name:  l_zhou
l_zhou  pass
For name:  h_jiang
h_jiang  pass
For name:  a_lewis
a_lewis  pass
For name:  c_meyer
c_meyer  pass
For name:  a_islam
a_islam  pass
For name:  k_fujita
k_fujita  pass
For name:  a_khan
a_khan  pass
For name:  a_kim
a_kim  pass
For name:  m_martinez
m_martinez  pass
For name:  m_aslam
m_aslam  pass
For name:  j_wolf
j_wolf  pass
For name:  s_agrawal
s_agrawal  pass
For name:  a_othman
a_othman  pass
For name:  k_evans
k_evans  pass
For name:  k_yoo
k_yoo  pass
For name:  d_turner
d_turner  pass
For name:  j_king
j_king  pass
For name:  b_shen
b_shen  pass
For name:  s_mishra
s_mishra  pass
For name:  c_o'connor
c_o'connor  pass
For name:  e_svensson
e_svensson  pass
For name:  o_ahmed
o_ahmed  pass
For name:  t_shimada
t_shimada  pass
For name:  a_watts
a_watts  pass
For name:  b_oliveira
b_oliveira  pass
For name:  t_ito
t_ito  pass
For name:  t_jackson
t_jackson  pass
For name:  m_romero
m_romero  pass
For name:  j_west
j_west  pass
For name

a_ellis  pass
For name:  f_reis
f_reis  pass
For name:  j_gray
j_gray  pass
For name:  r_hughes
r_hughes  pass
For name:  a_green
a_green  pass
For name:  c_reis
c_reis  pass
For name:  f_scott
f_scott  pass
For name:  l_han
l_han  pass
For name:  c_martins
c_martins  pass
For name:  r_schneider
r_schneider  pass
For name:  j_regan
j_regan  pass
For name:  s_brennan
s_brennan  pass
For name:  v_patel
v_patel  pass
For name:  d_johnston
d_johnston  pass
For name:  r_gupta
r_gupta  pass
For name:  s_reddy
s_reddy  pass
For name:  y_yao
y_yao  pass
For name:  a_huang
a_huang  pass
For name:  d_ghosh
d_ghosh  pass
For name:  r_morgan
r_morgan  pass
For name:  q_li
q_li  pass
For name:  w_wang
Total sample size before apply threshold:  765
Counter({'0000-0001-9033-0158': 194, '0000-0002-1430-1360': 101, '0000-0001-9093-412X': 39, '0000-0003-0509-2605': 30, '0000-0001-5983-3937': 29, '0000-0002-5369-5446': 28, '0000-0003-4287-1704': 27, '0000-0003-4053-5088': 24, '0000-0001-6022-1567': 21, '

Missing Sample:  25762636
Missing Sample:  15360326
Missing Sample:  23226803
Missing Sample:  24301072
Missing Sample:  25993365
Missing Sample:  19994868
Missing Sample:  26864734
Missing Sample:  26694251
Missing Sample:  26545749
Missing Sample:  24381794
Missing Sample:  26862851
Missing Sample:  27573925
Missing Sample:  18651769
Missing Sample:  26442953
Missing Sample:  27049720
Missing Sample:  26335618
Missing Sample:  26088413
Missing Sample:  23232914
Missing Sample:  21270524
Missing Sample:  21906091
Missing Sample:  24163332
Missing Sample:  26603572
Missing Sample:  25992645
Missing Sample:  27192127
Missing Sample:  19182891
Missing Sample:  27437873
Missing Sample:  25607240
Missing Sample:  26504932
Missing Sample:  27460659
Missing Sample:  20617322
Missing Sample:  24255003
Missing Sample:  27331625
Missing Sample:  25596741
Missing Sample:  26859895
Missing Sample:  22791448
Missing Sample:  27172220
Total missing sample:  36
Citation embedding shape:  (536, 100)


['0000-0003-1232-5307', '0000-0003-1835-9436', '0000-0003-3477-1172']
Total author after apply threshoid:  3
Total sample size after apply threshold:  470
Total missing sample:  0
Text embedding shape:  (0, 0)
Missing Sample:  11529720
Missing Sample:  12608813
Missing Sample:  24557432
Missing Sample:  11925252
Missing Sample:  12919039
Missing Sample:  12918967
Missing Sample:  22123305
Missing Sample:  18182398
Missing Sample:  11975590
Missing Sample:  21791181
Missing Sample:  23869171
Missing Sample:  12201775
Missing Sample:  22355459
Missing Sample:  19071448
Missing Sample:  22850941
Missing Sample:  15762711
Missing Sample:  23389421
Missing Sample:  18969699
Missing Sample:  21546871
Missing Sample:  15106978
Missing Sample:  18681453
Missing Sample:  17230295
Total missing sample:  22
Citation embedding shape:  (470, 100)
Final feature (combined embedding) shape:  (470, 100)
Multi-Class case
                     precision    recall  f1-score   support

0000-0003-1232-5307  

In [ ]:
print(pp_text_emb)
print(lr_diff_embedding_result)
print(svm_diff_embedding_result)
print(threshold_change)

In [ ]:
# combine n2v with textual embedding result together
embedding_type = ['n2v']
n2v_lr = [0.47620634494724273, 0.7498277720392034, 0.8281740036708996, 0.8647698255831765, 0.9028474372925239, 0.9201260416510354, 0.934862248014238, 0.9368194927355292, 0.9488163041946321, 0.9567160856788859]
n2v_svm = [0.5345640404538299, 0.7711647264987531, 0.8389260785222713, 0.8659119023637266, 0.901553749220774, 0.9167757971008624, 0.9329983382418031, 0.9334481127313246, 0.9430090615079809, 0.9509157091525307]

combined_emb = pp_text_emb+embedding_type

combined_lr_result = lr_diff_embedding_result
combined_lr_result.append(n2v_lr)

combined_svm_result = svm_diff_embedding_result
combined_svm_result.append(n2v_svm)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
threshold_change = np.array(threshold_change)
#--------------   logistic regression --------------------------#
# process result into np array
logistic_regression_result = np.array(combined_lr_result)
fig = plt.figure()
ax = plt.axes()
for emb_type, result in zip(combined_emb, logistic_regression_result):
    plt.plot(threshold_change, result, label=emb_type)
plt.legend()
plt.title('Average f1 for different embedding in logistic regression')
plt.xlabel('Threshold')
plt.ylabel('marco f1 score')

# plt.savefig('diff_embedding_sample=140k_clf=logistic regression.eps', format='eps', dpi=300)


# -------------------- svm -------------------------------------#
svm_result = np.array(combined_svm_result)
fig = plt.figure()
ax = plt.axes()
for emb_type, result in zip(combined_emb, svm_result):
    plt.plot(threshold_change, result, label=emb_type)
plt.legend()
plt.title('Average f1 for different embedding in SVM')
plt.xlabel('Threshold')
plt.ylabel('marco f1 score')

# plt.savefig('diff_embedding_sample=140k_clf=SVM.eps', format='eps', dpi=300)

In [ ]:
print(pp_text_emb)
print("svc: ", modelSVCf1)
print("lr: ", modelLRf1)

In [ ]:
# accuracy
from statistics import mean 
cleaned_svcLinear_accuracy = [x for x in all_svcLinear_accuracy if isinstance(x, float)]
cleaned_lr_accuracy = [x for x in all_LR_accuracy if isinstance(x, float)]
print(len(cleaned_svcLinear_accuracy))
print(len(cleaned_lr_accuracy))
print(mean(cleaned_svcLinear_accuracy))
print(mean(cleaned_lr_accuracy))

In [ ]:
# f1
from statistics import mean 
# remove string from result
cleaned_svcLinear_f1 = [x for x in all_svcLinear_f1 if isinstance(x, float)]
cleaned_lr_f1 = [x for x in all_LR_f1 if isinstance(x, float)]
print(len(cleaned_svcLinear_f1))
print(len(cleaned_lr_f1))
print(mean(cleaned_svcLinear_f1))
print(mean(cleaned_lr_f1))

In [ ]:
%reset

In [ ]:
%who